In [16]:
load('.RData')

In [17]:
library(ggplot2)
library(tidyverse)
library(ggpubr)
library(scatterplot3d)
library(reshape2)
library(MASS)
library(caret)
library(klaR)

In [18]:
fit_control <- trainControl(## 10-fold CV
                           method = "repeatedcv",
                           number = 10,
                           ## repeated ten times
                           repeats = 10,
                           classProbs=TRUE)

In [19]:
head(df)

,MouseID,Genotype,Treatment,Behavior,class,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,⋯,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,pS6_N,SYP_N,CaNA_N
,<chr>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,309_1,c,m,CS,cCSm,0.5036439,0.7471932,0.4301753,2.816329,5.990152,⋯,0.2475378,1.603310,2.014875,0.1082343,1.0449792,0.8315565,0.1888517,0.1063052,0.4270992,1.675652
2,309_2,c,m,CS,cCSm,0.5146171,0.6890635,0.4117703,2.789514,5.685038,⋯,0.2576322,1.671738,2.004605,0.1097485,1.0098831,0.8492704,0.2004036,0.1065922,0.4415813,1.743610
3,309_3,c,m,CS,cCSm,0.5091831,0.7302468,0.4183088,2.687201,5.622059,⋯,0.2553430,1.663550,2.016831,0.1081962,0.9968476,0.8467087,0.1936845,0.1083031,0.4357769,1.926427
4,309_4,c,m,CS,cCSm,0.4421067,0.6170762,0.3586263,2.466947,4.979503,⋯,0.2511031,1.484624,1.957233,0.1198832,0.9902247,0.8332768,0.1921119,0.1031838,0.3916910,1.700563
5,309_5,c,m,CS,cCSm,0.4349402,0.6174298,0.3588022,2.365785,4.718679,⋯,0.2517730,1.534835,2.009109,0.1195244,0.9977750,0.8786678,0.2056042,0.1047838,0.4341538,1.839730
6,309_6,c,m,CS,cCSm,0.4475064,0.6281758,0.3673881,2.385939,4.807635,⋯,0.2444853,1.507777,2.003535,0.1206872,0.9201782,0.8436793,0.1904695,0.1064762,0.4398331,1.816389


In [20]:
df.sc <- as.data.frame(scale(df[, 6:76]))
df.sc$class <- df$class

In [21]:
head(df.sc)

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,⋯,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,pS6_N,SYP_N,CaNA_N,class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.30141507,0.50112135,2.2587013,1.4855275,2.2936961,-0.3788304,-0.1951950,-0.8902474,0.5883106,0.67317357,⋯,0.19856160,-0.8731476,-2.124528,-1.131267,-0.167265548,-1.2822191,-1.0503864,-0.27223386,1.053565,cCSm
2,0.34495936,0.27207425,1.8877010,1.4091193,1.9693630,-0.5559283,-0.3769223,-0.9533663,0.4285810,0.34487863,⋯,0.47330772,-0.9133671,-2.062360,-1.333445,0.019042545,-0.8876409,-1.0304894,-0.05550198,1.265374,cCSm
3,0.32339597,0.43434770,2.0195004,1.1175771,1.9024166,-0.6205565,-0.2657387,-0.9601840,0.5281811,0.27040705,⋯,0.44043317,-0.8654873,-2.126094,-1.408539,-0.007900588,-1.1171427,-0.9118546,-0.14236800,1.835177,cCSm
4,0.05722063,-0.01157613,0.8164430,0.4899607,1.2193862,-0.2789844,-0.2374033,-1.0615492,-0.1787454,0.34213711,⋯,-0.27797789,-1.0988975,-1.646258,-1.446691,-0.149172515,-1.1708604,-1.2668294,-0.80213571,1.131207,cCSm
5,0.02878243,-0.01018251,0.8199886,0.2016973,0.9421321,-0.5197450,-0.3459253,-1.0756955,-0.6302974,0.14915112,⋯,-0.07637402,-0.8957298,-1.660991,-1.403196,0.328232736,-0.7100037,-1.1558848,-0.16665810,1.564962,cCSm
6,0.07864792,0.03215969,0.9930598,0.2591266,1.0366923,-0.3850403,-0.2461765,-1.0700726,-0.5381514,0.01781565,⋯,-0.18501586,-0.9175582,-1.613249,-1.850208,-0.039762642,-1.2269602,-1.0385268,-0.08166385,1.492211,cCSm


In [22]:
set.seed(99)
index <- createDataPartition(df.sc$class, p = 0.70, list = FALSE)
train_data <- df.sc[index, ]
test_data <- df.sc[-index, ]
sprintf("The trainining set contains %d rows", nrow(train_data))
sprintf("The test set contains %d rows", nrow(test_data))

[1] "The trainining set contains 735 rows"

[1] "The test set contains 312 rows"

In [25]:
#El modelo da error por variables coliniares.
#eliminamos las variables con alta correlacion
correlationMatrix <- cor(df.sc[, 1:71])
highlyCorrelated <- findCorrelation(correlationMatrix, cutoff=0.9)
highCorrelatedColumns <- colnames(correlationMatrix[, highlyCorrelated])
highCorrelatedColumns
train_data <- train_data[, !colnames(train_data) %in% c(highCorrelatedColumns, class)]
test_data <- test_data[, !colnames(test_data) %in% c(highCorrelatedColumns, class)]


[1] "NR1_N"    "pNR2B_N"  "ARC_N"    "pERK_N"   "DYRK1A_N" "BRAF_N"

In [28]:
table(train_data$class)
train_balanced <- upSample(x = train_data, y = train_data$class)
train_balanced <- train_balanced[, !(colnames(train_balanced) %in% c("Class"))]

table(train_balanced$class)


cCSm cCSs cSCm cSCs tCSm tCSs tSCm tSCs 
 105   84  105   84   95   74   95   93 


cCSm cCSs cSCm cSCs tCSm tCSs tSCm tSCs 
 105  105  105  105  105  105  105  105 

In [29]:
lda_model <- train(class ~ ., 
                   data = train_balanced, 
                   method = 'lda', 
                   trControl = fit_control)
lda_model

Linear Discriminant Analysis 

840 samples
 65 predictor
  8 classes: 'cCSm', 'cCSs', 'cSCm', 'cSCs', 'tCSm', 'tCSs', 'tSCm', 'tSCs' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 754, 758, 756, 756, 755, 757, ... 
Resampling results:

  Accuracy   Kappa   
  0.9810848  0.978377


In [30]:
lda_preds <- predict(lda_model, newdata = test_data)
confusionMatrix(lda_preds, test_data$class)

Confusion Matrix and Statistics

          Reference
Prediction cCSm cCSs cSCm cSCs tCSm tCSs tSCm tSCs
      cCSm   44    0    0    0    0    0    0    0
      cCSs    0   34    0    0    0    0    0    0
      cSCm    0    0   45    0    0    0    0    0
      cSCs    0    0    0   36    0    0    0    0
      tCSm    1    2    0    0   40    0    0    0
      tCSs    0    0    0    0    0   31    0    0
      tSCm    0    0    0    0    0    0   40    0
      tSCs    0    0    0    0    0    0    0   39

Overall Statistics
                                         
               Accuracy : 0.9904         
                 95% CI : (0.9722, 0.998)
    No Information Rate : 0.1442         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.989          
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: cCSm Class: cCSs Class: cSCm Class: cSC

In [87]:
# Warning message:
# “model fit failed for Fold01.Rep01: parameter=none Error in qda.default(x, grouping, ...) : 
#   some group is too small for 'qda'



In [31]:
qda_model <- train(class ~ ., 
                   data = train_balanced, 
                   method = 'qda', 
                   trControl = fit_control)
qda_model

Warning message:
“model fit failed for Fold01.Rep05: parameter=none Error in qda.default(x, grouping, ...) : rank deficiency in group tCSs
”
Warning message:
“model fit failed for Fold02.Rep08: parameter=none Error in qda.default(x, grouping, ...) : rank deficiency in group tCSs
”
Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Quadratic Discriminant Analysis 

840 samples
 65 predictor
  8 classes: 'cCSm', 'cCSs', 'cSCm', 'cSCs', 'tCSm', 'tCSs', 'tSCm', 'tSCs' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 756, 757, 757, 755, 757, 755, ... 
Resampling results:

  Accuracy   Kappa    
  0.9516009  0.9446771


In [32]:
qda_preds <- predict(qda_model, newdata = test_data)
confusionMatrix(qda_preds, test_data$class)

Confusion Matrix and Statistics

          Reference
Prediction cCSm cCSs cSCm cSCs tCSm tCSs tSCm tSCs
      cCSm   45    0    0    0    0    6    0    0
      cCSs    0   36    0    0    0    3    0    0
      cSCm    0    0   45    0    0    0    0    0
      cSCs    0    0    0   35    0    0    0    0
      tCSm    0    0    0    0   40    2    0    0
      tCSs    0    0    0    0    0   20    0    0
      tSCm    0    0    0    1    0    0   40    0
      tSCs    0    0    0    0    0    0    0   39

Overall Statistics
                                        
               Accuracy : 0.9615        
                 95% CI : (0.9338, 0.98)
    No Information Rate : 0.1442        
    P-Value [Acc > NIR] : < 2.2e-16     
                                        
                  Kappa : 0.9559        
                                        
 Mcnemar's Test P-Value : NA            

Statistics by Class:

                     Class: cCSm Class: cCSs Class: cSCm Class: cSCs
Sensiti